In [ ]:
!pip install scikit-learn pandas numpy matplotlib seaborn fairlearn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 38.1 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.3
    Uninstalling scipy-1.16.3:
      Successfully uninstalled scipy-1.16.3


In [ ]:
import pandas as pd
import numpy as np

# Working raw CSV for German credit data
url = "https://raw.githubusercontent.com/praisan/hello-world/master/german_credit_data.csv"
df = pd.read_csv(url)

df.head()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        1000 non-null   int64 
 1   Age               1000 non-null   int64 
 2   Sex               1000 non-null   object
 3   Job               1000 non-null   int64 
 4   Housing           1000 non-null   object
 5   Saving accounts   817 non-null    object
 6   Checking account  606 non-null    object
 7   Credit amount     1000 non-null   int64 
 8   Duration          1000 non-null   int64 
 9   Purpose           1000 non-null   object
 10  Risk              1000 non-null   object
dtypes: int64(5), object(6)
memory usage: 86.1+ KB


In [ ]:
#Basic cleaning and encoding
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

# Target: 1 = good credit, 0 = bad credit
df['Risk'] = df['Risk'].map({'good': 1, 'bad': 0})

# Sensitive attribute for fairness analysis
sensitive_feature = df['Sex']  # 'male' / 'female'

X = df.drop(columns=['Risk'])
y = df['Risk']

# Identify categorical and numeric columns
cat_cols = X.select_dtypes(include=['object']).columns.tolist()
num_cols = X.select_dtypes(exclude=['object']).columns.tolist()

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(drop="first", handle_unknown="ignore"), cat_cols),
        ("num", "passthrough", num_cols),
    ]
)

# Baseline classifier
clf = LogisticRegression(max_iter=1000)

model = Pipeline(steps=[("preprocess", preprocess),
                       ("clf", clf)])


In [ ]:
# Train and evaluate baseline performance
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, classification_report

X_train, X_test, y_train, y_test, s_train, s_test = train_test_split(
    X, y, sensitive_feature, test_size=0.3, random_state=42, stratify=y
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))
print("F1:", f1_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.74
ROC-AUC: 0.7591534391534392
F1: 0.8289473684210527
              precision    recall  f1-score   support

           0       0.61      0.37      0.46        90
           1       0.77      0.90      0.83       210

    accuracy                           0.74       300
   macro avg       0.69      0.63      0.64       300
weighted avg       0.72      0.74      0.72       300



/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
#Group performance by sensitive attribute
def group_metrics(y_true, y_pred, group):
    groups = pd.Series(group).unique()
    for g in groups:
        mask = (group == g)
        acc = accuracy_score(y_true[mask], y_pred[mask])
        print(f"Group = {g}, size = {mask.sum()}, accuracy = {acc:.3f}")

group_metrics(y_test.reset_index(drop=True),
              pd.Series(y_pred).reset_index(drop=True),
              s_test.reset_index(drop=True))


Group = female, size = 96, accuracy = 0.719
Group = male, size = 204, accuracy = 0.750


In [ ]:
#Fairness metrics with Fairlearn
from fairlearn.metrics import (
    MetricFrame,
    selection_rate,
    true_positive_rate,
    false_positive_rate
)

# Recompute predictions as Series aligned with X_test index
y_pred_series = pd.Series(y_pred, index=y_test.index)
s_test_series = s_test

metrics = {
    "accuracy": accuracy_score,
    "selection_rate": selection_rate,
    "tpr": true_positive_rate,
    "fpr": false_positive_rate,
}

mf = MetricFrame(
    metrics=metrics,
    y_true=y_test,
    y_pred=y_pred_series,
    sensitive_features=s_test_series
)

print("Overall metrics:")
print(mf.overall)

print("\nBy-group metrics:")
print(mf.by_group)


Overall metrics:
accuracy          0.740000
selection_rate    0.820000
tpr               0.900000
fpr               0.633333
dtype: float64

By-group metrics:
        accuracy  selection_rate      tpr       fpr
Sex                                                
female   0.71875        0.729167  0.84127  0.515152
male     0.75000        0.862745  0.92517  0.701754


In [ ]:
disp_tpr = mf.by_group["tpr"].max() - mf.by_group["tpr"].min()
disp_sr  = mf.by_group["selection_rate"].max() - mf.by_group["selection_rate"].min()

print("TPR disparity:", disp_tpr)
print("Selection rate disparity:", disp_sr)


TPR disparity: 0.08390022675736963
Selection rate disparity: 0.1335784313725491


In [ ]:
from fairlearn.reductions import ExponentiatedGradient, DemographicParity

# Need a plain sklearn estimator (not the pipeline) inside the reduction
base_estimator = LogisticRegression(max_iter=1000)

# Fit preprocessing separately, then fairness reduction on transformed data
X_train_transformed = preprocess.fit_transform(X_train)
X_test_transformed = preprocess.transform(X_test)

mitigator = ExponentiatedGradient(
    estimator=base_estimator,
    constraints=DemographicParity()
)

mitigator.fit(X_train_transformed, y_train, sensitive_features=s_train)

y_pred_mitigated = mitigator.predict(X_test_transformed)

# Compare metrics
print("Mitigated accuracy:", accuracy_score(y_test, y_pred_mitigated))

mf_mitigated = MetricFrame(
    metrics=metrics,
    y_true=y_test,
    y_pred=y_pred_mitigated,
    sensitive_features=s_test_series
)

print("\nMitigated overall:")
print(mf_mitigated.overall)
print("\nMitigated by-group:")
print(mf_mitigated.by_group)


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Mitigated accuracy: 0.7333333333333333

Mitigated overall:
accuracy          0.733333
selection_rate    0.826667
tpr               0.900000
fpr               0.655556
dtype: float64

Mitigated by-group:
        accuracy  selection_rate       tpr       fpr
Sex                                                 
female  0.677083        0.812500  0.873016  0.696970
male    0.759804        0.833333  0.911565  0.631579


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# Refit a logistic regression on transformed training data for inspection
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_transformed, y_train)

# Get feature names after preprocessing
ohe = preprocess.named_transformers_["cat"]
cat_feature_names = ohe.get_feature_names_out(cat_cols)
feature_names = np.concatenate([cat_feature_names, num_cols])

coef = lr.coef_[0]
feat_imp = pd.Series(coef, index=feature_names).sort_values(key=abs, ascending=False)

feat_imp.head(15)


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,0
Checking account_nan,1.656123
Checking account_rich,0.817265
Saving accounts_rich,0.797054
Saving accounts_nan,0.772626
Purpose_education,-0.745479
Purpose_radio/TV,0.539236
Saving accounts_quite rich,0.484583
Sex_male,0.339697
Checking account_moderate,0.309612
Purpose_repairs,-0.265100
